# Creating your own GAN III: LR-GAN

In the last notebook we implemented the `Pix2Pix` GAN together, this time we're tackling the Latent-Regressor GAN. It is described in one of our favorite papers describing a really amazing algorithm: The [BicycleGAN](https://arxiv.org/pdf/1711.11586.pdf). This will soon be implemented as well in `vegans`. As the time of writing this notebook (2021-04-08 20:22, only one hour passed since we started implementing the Pix2Pix :) ) there are only 4 (9) GAN architectures implemented in `vegans`: `VanillaGAN`, `WasssersteinGAN`, `WassersteinGANGP`, `LSGAN` and all there conditional variants plus the Pix2Pix which only is a conditional algorithm. The LR-GAN finally tackles the problem of mode collapse which means that all random inputs into the generator are mapped to a single output which might fool the discriminator perfectly but does not look like a real image to humans. Even if i does it is not desirable as we want a high variety of output images. This is one of the most pressing problems so it is good that we finally deal with it (somewhat).

We suppose you have read the previous notebooks on the creation of the `LSGAN` and `Pix2Pix`. If not, please go over it so you have a understanding of the abstract base classes. We will not present it here and jump basically right into the implementation.

First import the usual libraries:

In [1]:
import os
import torch

from vegans.GAN import ConditionalWassersteinGAN, ConditionalWassersteinGANGP
from vegans.utils.utils import plot_losses, plot_images, get_input_dim

## Latent-Regressor GAN

Remember that so far we only looked at two networks learning against each other. A discriminator which is trained to differentiate between real and fake images and a generator which is trained to fool the discriminator. As mentioned above this quite often leads to mode collapse where the generator produces a single "perfect" image regardless of input. In the well known example of generating handwritten digits, the generator might learn to produce the perfect image of a zero but produces nothing else. The discriminator can never tell if the generator images are real or fake so the generator is content and will stop learning. The latent regressor GAN approaches this problem elegantly by introducing another helper network (Note that another technique to deal with this is minibatch discrimination which is not yet supported in `vegans`, but hopefully is in the future (it might even be implemented by the time you read this)).

The job of this helper network is to take the output of the generator (often a generated image) and compress it back into the latent space. So if we started from a random image (say with shape [1, 4, 4]) the generator produces an image from it and the helper network maps at back to a space of the initial dimension (again [1, 4, 4]). For this reason this network is called an **Encoder**. 
The output of the encoder is then compared to the initial latent input to the generator and a L1 (L2) norm is computed between the two. The goal of the generator (and encoder) is to minimise this L1 (L2) norm. That's why it is called a Latent-Space Regressor.

This helps against mode collapse because if every latent input to the generator is mapped to basically the same output image the encoder will transform all those images back to one single latent vector. This vector will most of the times be quite different from the original input so the L1 norm increases which the generator has to minimize. Therefore it needs to produce a reproducible output for every input. The schema looks like this (again taken from the paper for BiCycleGAN):

![LRGANSchema](./picts/LRGANSchema.png)

Let's now start with the implementation of the LRGAN. First note that unlike before we can't use the parent class `AbstractGAN1v1` because we now have three networks working against each other. There is no base class for this case in `vegans` (yet), so we can only inherit from `AbstractGenerativeModel` and do a lot of the footwork on our own. This means we need to implement all abstract methods:

- \_\_init\_\_(self, x_dim, z_dim, optim, optim_kwargs, fixed_noise_size, device, folder, ngpu):
    This takes care of the initializaton and the method 
    
    super().\_\_init\_\_(
        x_dim=x_dim, z_dim=z_dim, optim=optim, optim_kwargs=optim_kwargs,
        fixed_noise_size=fixed_noise_size, device=device, folder=folder, ngpu=ngpu
     )
     
     must be called at the end of the `__init__` method.

- _define_loss(self): Not strictly necessary but it is still kept as an abstract method so that the user has to think about what he wants to implement here. You can also implement it with a single `pass` statement. However, we will show you it's intended use here.

- calculate_losses(self, X_batch, Z_batch, who): The core function that needs to be implemented. For every batch it must populate an already existing (but empty) dictionary `self._losses`. The keys of the dictionary must include at least the keys used in `self.neural_nets` (explained below), but can also contain other losses. We will discuss this further in later implementations.

We start with the `\_\_init\_\_` method which **MUST** create the self.neural_nets dictionary. No worries, if you forget to specify it the `AbstractGenerativeModel` class will kindly remind you to populate the dictionary. I will copy some of the code from [AbstractGAN1v1](https://github.com/tneuer/GAN-pytorch/blob/main/vegans/models/unconditional/AbstractGAN1v1.py) just to get a feeling for how to start. I will reuse some of the code of course because that's what past-me would have wanted.

In [2]:
from vegans.models.unconditional.AbstractGenerativeModel import AbstractGenerativeModel
from torch.nn import BCELoss, L1Loss
from torch.nn import MSELoss as L2Loss
from vegans.utils.networks import Generator, Adversary, Encoder

In [4]:
class LRGAN(AbstractGenerativeModel):
    #########################################################################
    # Actions before training
    #########################################################################
    def __init__(
            self,
            generator,
            adversary,
            encoder,
            x_dim,
            z_dim,
            optim=None,
            optim_kwargs=None,
            lambda_z=10,
            feature_layer=None,
            fixed_noise_size=32,
            device=None,
            folder="./AbstractGAN1v1",
            ngpu=0):

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.generator = Generator(generator, input_size=z_dim, device=device, ngpu=ngpu)
        self.adversary = Adversary(adversary, input_size=x_dim, adv_type="Discriminator", device=device, ngpu=ngpu)
        self.encoder = Encoder(encoder, input_size=x_dim, device=device, ngpu=ngpu)
        self.neural_nets = {
            "Generator": self.generator, "Adversary": self.adversary, "Encoder": self.encoder
        }

        AbstractGenerativeModel.__init__(
            self, x_dim=x_dim, z_dim=z_dim, optim=optim, optim_kwargs=optim_kwargs, feature_layer=feature_layer,
            fixed_noise_size=fixed_noise_size, device=device, folder=folder, ngpu=ngpu
        )
        self.lambda_z = lambda_z
        self.hyperparameters["lambda_z"] = lambda_z

    def _define_loss(self):
        self.loss_functions = {"Generator": BCELoss(), "Adversary": BCELoss(), "L1": L1Loss()}
        

I included the construction of three `vegans` classes in the vegans.utils.networks module: Generator, Adversary, Encoder. If you are sneaky and look at the [code](https://github.com/tneuer/GAN-pytorch/blob/main/vegans/utils/networks.py) you will notice that there is absolutely no difference between the implementations of Generator and Encoder (apart from the `name` attribute which only shows up when printing the network). On an abstract level both do the same thing: take an input (image or vector), applying weights and biases and finally producing an output (image or vector). Everything else is implementation detail. We also already implemented the `_define_loss` method which includes the L1 loss for the encoder and generator. We also included the parameter `lambda_z` which will be later used to weight the L1 metric in the generator loss.

If you have worked through the previous notebooks you might note that the names in the self.neural_nets dictionary are extremely important! They will be from now in used in every relevant dictionary (`self.optimizers`, `self.steps`, `self._losses` to refer to this network. Give it a descriptive but concise name).

Next we will implement the last abstract method: calculate_losses(...)! I again copy and modify existing code from the [AbstractGAN1v1](https://github.com/tneuer/GAN-pytorch/blob/main/vegans/models/unconditional/AbstractGAN1v1.py) class.

In [8]:
class LRGAN(AbstractGenerativeModel):
    #########################################################################
    # Actions before training
    #########################################################################
    def __init__(
            self,
            generator,
            adversary,
            encoder,
            x_dim,
            z_dim,
            optim=None,
            optim_kwargs=None,
            lambda_z=10,
            feature_layer=None,
            fixed_noise_size=32,
            device=None,
            folder="./AbstractGAN1v1",
            ngpu=0):

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.generator = Generator(generator, input_size=z_dim, device=device, ngpu=ngpu)
        self.adversary = Adversary(adversary, input_size=x_dim, adv_type="Discriminator", device=device, ngpu=ngpu)
        self.encoder = Encoder(encoder, input_size=x_dim, device=device, ngpu=ngpu)
        self.neural_nets = {
            "Generator": self.generator, "Adversary": self.adversary, "Encoder": self.encoder
        }

        AbstractGenerativeModel.__init__(
            self, x_dim=x_dim, z_dim=z_dim, optim=optim, optim_kwargs=optim_kwargs, feature_layer=feature_layer,
            fixed_noise_size=fixed_noise_size, device=device, folder=folder, ngpu=ngpu
        )
        self.lambda_z = lambda_z
        self.hyperparameters["lambda_z"] = lambda_z

    def _define_loss(self):
        self.loss_functions = {"Generator": BCELoss(), "Adversary": BCELoss(), "L1": L1Loss()}
        
        
    #########################################################################
    # Actions during training
    #########################################################################
    def encode(self, x):
        return self.encoder(x)
    
    def calculate_losses(self, X_batch, Z_batch, who=None):
        if who == "Generator":
            self._calculate_generator_loss(X_batch=X_batch, Z_batch=Z_batch)
        elif who == "Adversary":
            self._calculate_adversary_loss(X_batch=X_batch, Z_batch=Z_batch)
        elif who == "Encoder":
            self._calculate_encoder_loss(X_batch=X_batch, Z_batch=Z_batch)
        else:
            self._calculate_generator_loss(X_batch=X_batch, Z_batch=Z_batch)
            self._calculate_adversary_loss(X_batch=X_batch, Z_batch=Z_batch)
            self._calculate_encoder_loss(X_batch=X_batch, Z_batch=Z_batch)
            

    def _calculate_generator_loss(self, X_batch, Z_batch, y_batch):
        fake_images = self.generate(y=y_batch, z=Z_batch)
        encoded_space = self.encode(x=fake_images, y=y_batch)

        if self.feature_layer is None:
            fake_predictions = self.predict(x=fake_images, y=y_batch)
            gen_loss_original = self.loss_functions["Generator"](
                fake_predictions, torch.ones_like(fake_predictions, requires_grad=False)
            )
        else:
            gen_loss_original = self._calculate_feature_loss(X_real=X_batch, X_fake=fake_images, y_batch=y_batch)
        latent_space_regression = self.loss_functions["L1"](
            encoded_space, Z_batch
        )
        gen_loss = gen_loss_original + self.lambda_z*latent_space_regression
        self._losses.update({
            "Generator": gen_loss,
            "Generator_Original": gen_loss_original,
            "Generator_L1": self.lambda_z*latent_space_regression
        })

    def _calculate_adversary_loss(self, X_batch, Z_batch):
        fake_images = self.generate(z=Z_batch).detach()
        fake_predictions = self.predict(x=fake_images)
        real_predictions = self.predict(x=X_batch.float())

        adv_loss_fake = self.loss_functions["Adversary"](
            fake_predictions, torch.zeros_like(fake_predictions, requires_grad=False)
        )
        adv_loss_real = self.loss_functions["Adversary"](
            real_predictions, torch.ones_like(real_predictions, requires_grad=False)
        )
        adv_loss = 0.5*(adv_loss_fake + adv_loss_real)
        self._losses.update({
            "Adversary": adv_loss,
            "Adversary_fake": adv_loss_fake,
            "Adversary_real": adv_loss_real,
            "RealFakeRatio": adv_loss_real / adv_loss_fake
        })
        
    def _calculate_encoder_loss(self, X_batch, Z_batch):
        fake_images = self.generate(z=Z_batch).detach()
        encoded_space = self.encoder(fake_images)
        latent_space_regression = self.loss_functions["L1"](
            encoded_space, Z_batch
        )
        self._losses.update({
            "Encoder": latent_space_regression
        })

This should do the trick and wasn't to complicated. To implement the conditional version allow for `y_dim`, `y_batch` and concatenate at the correct positions. Look at the source code to see that it is almost the same :)

Please again note that this is a prelimanary tutorial implementation which might or might not change in future releases of `vegans`. So this implementation might not be completely up-to-date, but still is a viable implementation nonetheless.